## Autogeneration of license plates images for training LPRNet

This notebook provide a skeleton to generate random license plate images for training LPRNet for license plate recognition. Auto-generation of synthetic training data constitutes an easy strategy of generating large datasets in a cheap manner, alleviating the burden of collecting and annotating real data. In contrast to many other computer vision tasks, training a network for text recognition can rely on synthetic data alone and achieve a good recognition performance on real-life data. 

In order to use thiss notebook, clean license plate templates (with no text) and predefined text fonts (`.ttf`) for this purpose should be prepared in advance and placed under the following paths:
 - `./plates/` - path to clean license plates:
 - `./fonts` - path to clean license plates

<br>
There are several knobs that can be tempered with to control the characteristics of the generated license plates, such as:
 
 - license plate text length
 - coordinates within the plate to write the text at
 - spereations between the license plate text characters
 - color difference between each character in the license plate

**_Notes_**:
 - The current flow is designed to generate license plates for training the model to recognize Israeli license plates. 
 - Hailo's LPRNet was trained on an autogenearted dataset of 4 million images. Training the model on smaller datasets resulted in a significant accuracy drop
 - In addition to the large amount of training data, we have also used data augmentations in the training phase, which we found to greatly aid both recongnition performance and training convergence.

In [ ]:
#imports
import random
import os
import numpy as np
from IPython.display import display
from IPython.display import Image as Display
from PIL import ImageDraw, ImageFont, Image
import imgaug.augmenters as iaa
import cv2
import time
import multiprocessing
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
# global configurations
clean_plates = [os.path.join('plates', x) for x in os.listdir('plates')]
fonts = [os.path.join('fonts', x) for x in os.listdir('fonts')]

font_size = [40, 60]               # font size min,max
plate_length = [7, 8]              # plate length min,max
basic_color_diff = 100             # difference in color form black/white
right_margin = [35, 45]            # margin to start the text from
top_margin = [8, 12]               # margin to start the text from
margin_between_letters = [10, 25]  # margin between two letters


# per letter configurations
difference_in_size = 5    # font size diff between letters
difference_in_color = 50  # color diff between letters
difference_in_loc = 5     # diff in location between letters

In [ ]:
# define some util functions

# for each clean plate, define the (relative) box coords to be distorted
box_coord_to_distort = {0: [0.115, 0.97, 0.13, 0.90],
                        1: [0.115, 0.945, 0.18, 0.85],
                        2: [0.105, 0.98, 0.08, 0.92],
                        3: [0.13, 0.96, 0.18, 0.85],
                        4: [0.04, 0.96, 0.2, 0.80],
                        5: [0.13, 0.96, 0.15, 0.77],
                        }

def distort_background(img, xp0, xp1, yp0, yp1,
                       shape=(1000, 221),
                       mean_c = (175, 140, 55),
                       std_c = (10, 15, 40),
                       wfact=30, hfact=60
                       ):
    '''
    function gets a clean license plate image and relative box coordinates within it and distorts this area
    Args:
        img: clean license plate image
        xp0, xp1, yp0, yp1: relative box coordinates defining the crop to be distorted
        shape: resize the image to this value. box coordinates should be compatible with this value
        mean_c, std_c: define the min/max color value range to be sampled from and added
                       to each RGB channel in the cropped area
                       such that the min value is mean_c-std_c
                       and the max value is mean_c-std_c
        wfact, hfact: used to also downscale and upscale the cropped area dimensions to "pixelize" the distortion
    '''
    
    # resize image
    img = np.array(Image.fromarray(np.squeeze(img)).resize((shape)), dtype=np.uint8)
    
    # get absolute box coords in image to crop out and distort               
    xmin, xmax, ymin, ymax =\
        int(xp0*img.shape[1]), int(xp1*img.shape[1]),\
        int(yp0*img.shape[0]), int(yp1*img.shape[0])
    
    cut = img[ymin:ymax,xmin:xmax, :]
    new_width = int(cut.shape[0]/wfact)
    new_height = int(cut.shape[1]/hfact)

    # downsample cropped area to distort
    cut_resized = np.array(Image.fromarray(np.squeeze(cut)).resize((new_width, new_height)), dtype=np.uint8)
    h,w,c = cut_resized.shape
    distort = np.zeros((h,w,c), dtype=np.uint8)

    # add color distortion
    for i in range(distort.shape[-1]):
        distort[:,:,i] += np.random.randint(mean_c[i]-std_c[i], mean_c[i]+std_c[i], size=(h, w), dtype=np.uint8)
    
    new_img = img.copy().transpose((2,0,1))

    # upsample distorted area back to original shape, and plug into image
    new_img[:,ymin:ymax,xmin:xmax] = np.array(Image.fromarray(np.squeeze(distort)).resize(cut.shape[:2],Image.NEAREST), dtype=np.uint8).transpose()
    new_img = new_img.transpose((1,2,0))
    
    return Image.fromarray(np.squeeze(new_img))


def get_random_string():
    '''
    get the random license plate number
    '''
    def get_random_string_v1():
        lp_len = random.choice(plate_length)
        chars = [str(x) for x in range(10)]
        sep = ['-',' ']
        groups = []
        if lp_len == 7:
            groups.extend(random.choices(chars, k=2))
            groups.extend(random.choices(sep,weights=[85,15], k=1))
            groups.extend(random.choices(chars, k=3))
            groups.extend(random.choices(sep,weights=[85,15], k=1))
            groups.extend(random.choices(chars, k=2))
        elif lp_len == 8:
            groups.extend(random.choices(chars, k=3))
            groups.extend(random.choices(sep,weights=[85,15], k=1))
            groups.extend(random.choices(chars, k=2))
            groups.extend(random.choices(sep,weights=[85,15], k=1))
            groups.extend(random.choices(chars, k=3))
        return groups

    def get_random_string_v2():
        lp = random.choice(plate_length)
        num = [str(x) for x in range(10)]
        t = []
        for i in range(lp+2):
            if (((i == 3 or i == 6) and lp == 8) or
                ((i == 2 or i == 6) and lp == 7)):
                t.append(random.choice([' ', '-', '.']))
            else:
                t.append(random.choice(num))
        return t
    
    def get_random_string_v3():
        lp_len = random.choice(plate_length)
        chars = [str(x) for x in range(10)] + [' ', '-']
        p = [0.9 / 11] * 11 + [0.1]
        return np.random.choice(chars, lp_len, p=p)
    
    random_string_funcs = [get_random_string_v1,
                           get_random_string_v2,
                           get_random_string_v3]

    return np.random.choice(random_string_funcs)()

def write_text(img, text_list):
    '''
    Randomly add the generated plate number (text_list) into the clean plate (img)
    '''
    img_pil = Image.fromarray(img)
    font_sz = int(random.choice(np.linspace(font_size[0], font_size[1], 1 + font_size[1] - font_size[0])))
    basic_color = [random.choice(np.linspace(0, basic_color_diff, basic_color_diff+1)) for i in range(3)]
    fontc = random.choice(fonts)
    draw = ImageDraw.Draw(img_pil)
    w, h = img_pil.size
    rmargin = int(random.choice(np.linspace(right_margin[0], right_margin[1])))
    tmargin = int(random.choice(np.linspace(top_margin[0], top_margin[1])))
    f = ImageFont.truetype(fontc, font_sz)
    while True:
        tw, th = draw.textsize(''.join(text_list), font=f)
        if tw < w * 0.8:
            break
        font_sz -= 1
        f = ImageFont.truetype(fontc, font_sz)

    margin = tw / len(text_list)*1.05
    for i, t in enumerate(text_list):
        f = ImageFont.truetype(fontc, font_sz + random.choice([-1, 1]) * random.choice([x for x in range(difference_in_size)]))
        color = [int(x + random.choice([-1, 1]) * random.choice([x for x in range(difference_in_color)])) for x in basic_color]
        draw.text((rmargin + i * margin + random.choice([-1, 1]) * random.choice([x for x in range(difference_in_loc)]),
                   tmargin + random.choice([-1, 1]) * random.choice([x for x in range(difference_in_loc)])),
                  t, fill=tuple(color), font=f)
    return np.array(img_pil, np.uint8)

def gen_random_plate(shape=(1000, 221), final_shape=(300, 75)):
    '''
    get an autogenerated license plate image and its random plate number
    '''
    
    # get random plate number
    lp = get_random_string()
    lp_text = ''.join(lp).replace('.','').replace(' ','').replace('-','').replace('~','')

    # get clean plate
    clean_plate = random.choice(clean_plates)
    img = Image.open(clean_plate)
    
    if random.random() >= 0.5:
        # distort a certain area of the plate's background
        img = distort_background(img, *box_coord_to_distort[clean_plates.index(clean_plate)])
    
    img = Image.fromarray(np.squeeze(img)).resize(final_shape)

    # write the text
    img_pil = write_text(np.array(img), lp)
    img = np.array(img_pil)
    
    if random.random() >= 0.5:
        # convert to BW
        img = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    return img, lp_text


In [ ]:
# display some autogenerated examples

num_to_display = 20
lp_size = (300, 75) # (width x height)

for _ in range(num_to_display):
    img, seq = gen_random_plate(final_shape=lp_size)
    img = Image.fromarray(img)
    print(seq)
    display(img)

In [ ]:
# create images with multi_process
import time
import multiprocessing
from tqdm import tqdm

# autogenerated license plate image size (width x height)
lp_size = (300, 75)

def create_images(root_dir, i, k):
    root_dir = root_dir.format(i)
    if not os.path.exists(root_dir):
        os.mkdir(root_dir)
    for _ in tqdm(range(k)):
        img, seq = gen_random_plate(final_shape=lp_size)
        img = Image.fromarray(img)
        img.save(root_dir + seq + '.png')
    
root_dir = "./lp_autogenerate/train_batch{}/"
nproc = 16
train_size = 4000

start = time.perf_counter()
processes = []
for i in range(nproc):
    p = multiprocessing.Process(target=create_images, args = [root_dir, i, int(train_size/nproc)])
    p.start()
    processes.append(p)
for p in processes:
    print(F"{p}")
    p.join()
end = time.perf_counter()

print(f'Finished in {round(end-start, 2)} second(s)')